In [1]:
import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/anderson/.pyenv/versions/3.10.14/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-08 21:25:09.535886: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 21:25:09.891286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 21:25:10.588060: W tensorflow/compil

# Model

In [3]:
MODEL_ID = "KRAFTON/KORani-v3-13B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,
    device_map="auto",
    offload_folder="./offload",
    llm_int8_enable_fp32_cpu_offload=True,
    low_cpu_mem_usage=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/anderson/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:494: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pas_token_id` explicitly by `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation, and ensure your `input_ids` input does not have negative values.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

# Store Embedding 생성 및 Faiss 에 Indexing 

In [53]:
STORE_TEXTS = [
    "네네치킨 품동점. 가장 인기 있는 제품은 후라이드, 반반치킨이며, 그외 제품은 네네스위틱, 닭날개, 윙봉 있습니다",
    "후라이드 참 잘하는 집 풍동 식사점. 가장 인기 있는 제품은 매운양념치킨, 눈꽃 치즈치킨이며, 그외 제품은 허니버터치킨, 간장치킨 있습니다",
    "굽네치킨&피자 풍동점. 가장 인기 있는 제품은 간장치킨, 고추바사삭, 후라이드이며, 그외 제품은 웨지감자 있습니다",
    "처갓집양념치킨 백석점. 가장 인기 있는 제품은 순살 슈프림양념치킨, 반반치킨이며, 그외 제품은 매운양념치킨 있습니다",
    "이도돈까스. 가장 인기 있는 제품은 이도돈까스, 치즈돈까스이며, 그외 제품은 가께우동 있습니다",
    "돈까스왕. 가장 인기 있는 제품은 동까스왕이며, 그외 제품은 치즈까스 있습니다",
    "시골김치찌개. 가장 인기 있는 제품은 시골 김치찌개이며, 그외 제품은 도시락 있습니다",
    "전통김치찌개. 가장 인기 있는 제품은 전통김치찜, 그외 제품은 달걀말이 있습니다.",
]


def create_embeddings(model, texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        # (batch, word size, d_model).
        embeddings = model(**inputs)
        return embeddings.logits.mean(dim=1).cpu().numpy()


store_embeddings = create_embeddings(model, STORE_TEXTS)
print("store_embeddings:", store_embeddings.shape)

store_embeddings: (8, 32001)


In [54]:
# Faiss 인덱스 생성
index = faiss.IndexFlatL2(store_embeddings.shape[1])  # 32001 dimension
index.add(store_embeddings)

# Cosine Similarity

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

query = '네네치킨'

query_embedding = create_embeddings(model, [query])
print("query_embedding:", query_embedding.shape)

scores = cosine_similarity(store_embeddings, query_embedding)
ranks = np.argsort(-scores.reshape(-1))

for r in ranks:
    s = scores.reshape(-1)[r]
    print(f'{s:.4f} {STORE_TEXTS[r]}')

query_embedding: (1, 32001)
0.8117 네네치킨 품동점. 가장 인기 있는 제품은 후라이드, 반반치킨이며, 그외 제품은 네네스위틱, 닭날개, 윙봉 있습니다
0.7769 처갓집양념치킨 백석점. 가장 인기 있는 제품은 순살 슈프림양념치킨, 반반치킨이며, 그외 제품은 매운양념치킨 있습니다
0.7760 굽네치킨&피자 풍동점. 가장 인기 있는 제품은 간장치킨, 고추바사삭, 후라이드이며, 그외 제품은 웨지감자 있습니다
0.7631 돈까스왕. 가장 인기 있는 제품은 동까스왕이며, 그외 제품은 치즈까스 있습니다
0.7626 이도돈까스. 가장 인기 있는 제품은 이도돈까스, 치즈돈까스이며, 그외 제품은 가께우동 있습니다
0.7595 시골김치찌개. 가장 인기 있는 제품은 시골 김치찌개이며, 그외 제품은 도시락 있습니다
0.7451 후라이드 참 잘하는 집 풍동 식사점. 가장 인기 있는 제품은 매운양념치킨, 눈꽃 치즈치킨이며, 그외 제품은 허니버터치킨, 간장치킨 있습니다
0.7428 전통김치찌개. 가장 인기 있는 제품은 전통김치찜, 그외 제품은 달걀말이 있습니다.


# Search

In [59]:
query_embedding = create_embeddings(model, [query])
print("query_embedding:", query_embedding.shape)

distances, indices = index.search(query_embedding, k=3)

for d, i in zip(distances[0], indices[0]):
    print(f"{d:.0f}: {STORE_TEXTS[i][:20]}")

query_embedding: (1, 32001)
19297: 네네치킨 품동점. 가장 인기 있는 제
21548: 처갓집양념치킨 백석점. 가장 인기 있
22561: 굽네치킨&피자 풍동점. 가장 인기 있
